In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs
# Uncomment the following line if you are using TPU:
#!pip install tensorflow==2.14    ### because of the error when "from transformers import pipeline". This is only on TPU!

In [ ]:
# To run the training on TPU, you will need to uncomment the following line:
#!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl   ### IT DOES NOT WORK!
### CAN NOT IMPORT DataCollatorForLanguageModeling after the following line:
#!pip install cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

In [ ]:
from huggingface_hub import notebook_login, Repository, get_full_repo_name
import torch
from collections import defaultdict
from tqdm import tqdm
from tqdm.notebook import tqdm
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, DataCollatorForLanguageModeling, Trainer, TrainingArguments, pipeline, get_scheduler
from transformers import StoppingCriteriaList, StoppingCriteria
from torch.nn import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import IterableDataset
from torch.optim import AdamW
from accelerate import Accelerator
from transformers.keras_callbacks import PushToHubCallback

from google.colab import userdata
import json
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git config --global user.email "orest.andrusyshyn@ucu.edu.ua"
!git config --global user.name "Orest Andrusyshyn"
notebook_login()



### TRAINING WITH FIT

In [ ]:
!gzip -dkv pretrain_dataset_train.json.gz
!gzip -dkv pretrain_dataset_validation.json.gz

pretrain_dataset_train.json.gz:	 82.6% -- created pretrain_dataset_train.json
gzip: pretrain_dataset_validation.json already exists; do you wish to overwrite (y or n)? y
pretrain_dataset_validation.json.gz:	 80.5% -- created pretrain_dataset_validation.json


In [ ]:
# load dataset
data_files = {"train": "./pretrain_dataset_train.json", "validation": "./pretrain_dataset_validation.json"}
raw_datasets = load_dataset("json", data_files=data_files, field="data")
raw_datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['filepath', 'content'],
        num_rows: 5969
    })
    validation: Dataset({
        features: ['filepath', 'content'],
        num_rows: 663
    })
})

In [ ]:
context_length = 128
tokenizer = AutoTokenizer.from_pretrained("Andrusyshyn/gpt2-coq-tokenizer")

outputs = tokenizer(
    raw_datasets["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/845k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/490k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Input IDs length: 49
Input chunk lengths: [128, 128, 123, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 97]
Chunk mapping: [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

Map:   0%|          | 0/5969 [00:00<?, ? examples/s]

Map:   0%|          | 0/663 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 176596
    })
    validation: Dataset({
        features: ['input_ids'],
        num_rows: 21705
    })
})

In [ ]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

GPT-2 size: 125.8M parameters


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
gradient_accumulation_steps = 8
args = TrainingArguments(
    output_dir="gpt2-pretrained-for-coq-pt-local",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=1000//gradient_accumulation_steps,
    logging_steps=500//gradient_accumulation_steps,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=20//gradient_accumulation_steps,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=1000//gradient_accumulation_steps,
    fp16=True,
    hub_model_id="Andrusyshyn/gpt2-pretrained-for-coq-pt",
    push_to_hub=True,    # push to hub
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

In [ ]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
125,5.079600,5.077909
250,4.071200,4.133336
375,3.445700,3.612258
500,3.099000,3.336408
625,2.947500,3.227164


TrainOutput(global_step=689, training_loss=3.8474831408097887, metrics={'train_runtime': 2895.2595, 'train_samples_per_second': 60.995, 'train_steps_per_second': 0.238, 'total_flos': 1.1521933443072e+16, 'train_loss': 3.8474831408097887, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 3.219874382019043,
 'eval_runtime': 99.144,
 'eval_samples_per_second': 218.924,
 'eval_steps_per_second': 6.849,
 'epoch': 1.0}

In [ ]:
coq_model = GPT2LMHeadModel.from_pretrained("Andrusyshyn/gpt2-pretrained-for-coq-pt")
coq_tokenizer = AutoTokenizer.from_pretrained("Andrusyshyn/gpt2-coq-tokenizer")
pipe = pipeline(
    "text-generation", model=coq_model, tokenizer=coq_tokenizer, device=0
)

In [ ]:
txt = """Theorem identity : forall (n : nat), n = n."""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Theorem identity : forall (n : nat), n = n.
  Proof. 
    intros; intro; apply lt_lt_n_O_n.
  Qed.

  Lemma le_elt_add_1 : forall n, (s


### TRAINING WITH 🤗 ACCELERATE

#### HYPER PARAMS, GLOBAL VARS, FUNCTIONS

In [ ]:
# HYPER PARAMETERS
context_length = 1024
batch_size=4
weight_decay = 0.1
lr=5e-4
num_warmup_steps=50
gradient_accumulation_steps = 8
gradient_checkpointing = True
eval_steps = 1000 // gradient_accumulation_steps
num_train_epochs = 5

tokenizer_repo_name = "Andrusyshyn/gpt2-coq-tokenizer"
model_name = "gpt2-pretrained-for-coq-pt-custom-train"
model_output_dir = "./gpt2-pretrained-for-coq-pt-custom-train-local"
run_name = "cl_1024_tok_138304"
commit_hash = "91aac830c5ff8417d8bf389eea271a9d3dabab9c"
tokenizer_commit_hash = "b14705bd51541c24de6f62e974cb7a924a500220"

drive_train_res_path = '/content/gdrive/My Drive/UCU/diploma/progress_notes/training_results/second_training/theorems_cl_1024_tok_138304_train_res1.json'

partially_trained = False
previously_completed_steps = 2295
previous_global_steps = 18360
previous_step = 1836
stopped_lr = lr
stopped_epoch = 9


torch.manual_seed(7)

In [ ]:
# CUSTOM LOSS FUNCTION
def custom_weighted_loss(inputs, logits, alpha=1.0):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    # Calculate and scale weighting
    # weights = torch.stack([(inputs == kt).float() for kt in keytoken_ids]).sum(
    #     axis=[0, 2]
    # )
    # weights = alpha * (1.0 + weights)
    # print(loss_per_sample)
    weights = alpha * torch.ones_like(loss_per_sample)
    # print(weights)
    # Calculate weighted average
    weighted_loss = (loss_per_sample * weights).mean()
    return weighted_loss

In [ ]:
# PARAMETERS FOR WEIGHT DECAY
def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

In [ ]:
# TOKENIZE RAW DATASETS
def get_tokenized_dataset(p_raw_dataset, p_context_length, p_tokenizer):
  all_tokens_ids = []
  for sample in p_raw_dataset:
    tokenized_sample = p_tokenizer(sample["content"], truncation=False)["input_ids"]
    all_tokens_ids.extend(tokenized_sample + [p_tokenizer.eos_token_id])
  tokenized_dataset_list = []
  for i in range(0, len(all_tokens_ids), p_context_length):
      input_ids = all_tokens_ids[i : i + p_context_length]
      if len(input_ids) == p_context_length:
          tokenized_dataset_list.append(torch.tensor(input_ids))

  return Dataset.from_dict({"input_ids": tokenized_dataset_list})

In [ ]:
def save_results(filepath:str, split: str, results: dict):
    if split not in {"train", "valid"}:
        print("ERROR: INVALID SPLIT")
        return
    json_data = {"train": [], "valid": []}
    if os.path.exists(filepath):
        with open(filepath, 'r') as json_file:
            json_data = json.load(json_file)
    json_data[split].append(results)
    with open(filepath, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)

In [ ]:
# CONFIGURING GIT DIRECTORIES
repo_name = get_full_repo_name(model_name)
print(repo_name)
repo = Repository(model_output_dir, clone_from=repo_name)
repo.git_checkout(run_name, create_branch_ok=True)

Andrusyshyn/gpt2-pretrained-for-coq-pt-custom-train


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Andrusyshyn/gpt2-pretrained-for-coq-pt-custom-train into local empty directory.


Download file model.safetensors:   0%|          | 8.00k/480M [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/480M [00:00<?, ?B/s]

Revision `cl_1024_tok_138304` does not exist. Created and checked out branch `cl_1024_tok_138304`.



#### DATASETS

In [ ]:
# UNPACK DATASETS
!gzip -dkv pretrain_dataset_train.json.gz
!gzip -dkv pretrain_dataset_validation.json.gz

pretrain_dataset_train.json.gz:	 82.6% -- created pretrain_dataset_train.json
pretrain_dataset_validation.json.gz:	 80.5% -- created pretrain_dataset_validation.json


In [ ]:
# LOAD DATASETS
data_files = {"train": "./pretrain_dataset_train.json", "validation": "./pretrain_dataset_validation.json"}
# data_files = {"train": "./theorems_train.json", "validation": "./theorems_valid.json"}
raw_datasets = load_dataset("json", data_files=data_files, field="data")
raw_datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['filepath', 'content'],
        num_rows: 5969
    })
    validation: Dataset({
        features: ['filepath', 'content'],
        num_rows: 663
    })
})

In [ ]:
# DEBUG CODE (taking few samples from whole DatasetDict)
train_debug = raw_datasets["train"][:100]
valid_debug = raw_datasets["validation"][:3]
train_debug_dataset = Dataset.from_dict(train_debug)
valid_debug_dataset = Dataset.from_dict(valid_debug)
raw_datasets = DatasetDict({"train": train_debug_dataset, "validation": raw_datasets["validation"]})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['content', 'filepath'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['content', 'filepath'],
        num_rows: 12401
    })
})

In [ ]:
# LOADING TOKENIZER
tokenizer = AutoTokenizer.from_pretrained(tokenizer_repo_name, revision=tokenizer_commit_hash)
print("tokenizer len: ", len(tokenizer))
# TOKENIZE RAW DATASETS
train_dataset = get_tokenized_dataset(raw_datasets["train"],      context_length, tokenizer)
valid_dataset = get_tokenized_dataset(raw_datasets["validation"], context_length, tokenizer)
train_dataset.set_format("torch")
valid_dataset.set_format("torch")

# CREATE DATALOADERS
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader  = DataLoader(valid_dataset, batch_size=batch_size)
print(train_dataset)
print(valid_dataset)
print(len(train_dataloader))
print(len(eval_dataloader))

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (5832 > 1024). Running this sequence through the model will result in indexing errors


tokenizer len:  138304
Dataset({
    features: ['input_ids'],
    num_rows: 22313
})
Dataset({
    features: ['input_ids'],
    num_rows: 2721
})
5579
681


#### CONFIGURING MODEL AND TRAINING

In [ ]:
# PRE TRAINING MODEL FROM SCRATCH!
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
if gradient_checkpointing:
    model.gradient_checkpointing_enable()
optimizer = AdamW(get_grouped_params(model), lr=lr)

# if you’re training on a TPU, you’ll need to move all the code starting from here into a dedicated training function.
# See Chapter 3 for more details.
accelerator = Accelerator(mixed_precision="fp16")

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# PRE TRAINING MODEL FROM SCRATCH!
num_steps_per_epoch = len(train_dataloader)
print("Num training steps per epoch: ", num_steps_per_epoch)
num_training_completed_steps = (num_train_epochs * num_steps_per_epoch) // gradient_accumulation_steps
if ((num_train_epochs * num_steps_per_epoch) % gradient_accumulation_steps != 0):
  num_training_completed_steps += 1
print("Num total training completed steps: ", num_training_completed_steps)

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_completed_steps,    ### Wrong steps?
)

Num training steps per epoch:  5579
Num total training completed steps:  3487


In [ ]:
# USING ALREADY TRAINED MODEL!!!
num_steps_per_epoch = len(train_dataloader)
num_training_completed_steps = (num_train_epochs * num_steps_per_epoch) // gradient_accumulation_steps
if ((num_train_epochs * num_steps_per_epoch) % gradient_accumulation_steps != 0):
  num_training_completed_steps += 1

lr_sch_warmup_steps = num_warmup_steps
lr_sch_lr = lr
if partially_trained:
  lr_sch_warmup_steps = 0
  num_training_completed_steps -= previously_completed_steps
  lr_sch_lr = stopped_lr

print("Num training completed steps: ", num_training_completed_steps)
print("Start Learning rate: ", lr_sch_lr)
print("Warmup steps: ", lr_sch_warmup_steps)
################################################################################
model = GPT2LMHeadModel.from_pretrained(repo_name, revision=commit_hash)
if gradient_checkpointing:
    model.gradient_checkpointing_enable()
optimizer = AdamW(get_grouped_params(model), lr=lr_sch_lr)
accelerator = Accelerator(mixed_precision="fp16")
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=lr_sch_warmup_steps,
    num_training_steps=num_training_completed_steps,   ### Wrong steps?
)

Num training completed steps:  -2292
Start Learning rate:  0.0005
Warmup steps:  0


In [ ]:
# EVALUATION FUNCTION
def evaluate():
    torch.cuda.empty_cache()

    model.eval()
    losses = []
    with torch.no_grad():
      for step, batch in enumerate(eval_dataloader):
          # print(batch)
          with torch.no_grad():
              outputs = model(batch["input_ids"], labels=batch["input_ids"])
          # print(outputs.loss)
          # loss = outputs.loss.repeat(batch_size)
          # print(loss)
          # losses.append(accelerator.gather(loss))
          losses.append(outputs.loss)
          # break
    # print(losses)
    loss = torch.mean(torch.Tensor(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")

    torch.cuda.empty_cache()
    return loss.item(), perplexity.item()

In [ ]:
evaluate()

(1.183862328529358, 3.266968011856079)

In [ ]:
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 192.1M parameters


In [ ]:
# TRAINING LOOP

model.train()
completed_steps = 0    # effective batch step (steps = completed_steps * gradient_acuumulation_steps + remainder)
global_steps = 0
if partially_trained:
  completed_steps = previously_completed_steps
  global_steps = previous_global_steps
for epoch in range(num_train_epochs):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=0), total=num_steps_per_epoch
    ):
        if partially_trained and ((epoch<stopped_epoch) or ((epoch==stopped_epoch) and (step <= previous_step))):
          continue

        logits = model(batch["input_ids"]).logits
        loss = custom_weighted_loss(batch["input_ids"], logits)
################################################################################
        if (num_steps_per_epoch < 10) or (global_steps % ((int(0.1 * num_steps_per_epoch) // gradient_accumulation_steps) * gradient_accumulation_steps) == 0):
            log_train = {
                    "epoch": epoch,
                    "lr": lr_scheduler.get_lr(),
                    "steps": step,
                    "global_steps" : global_steps,
                    "competed_steps": completed_steps,
                    "loss/train": loss.item(),
            }
            accelerator.print(log_train)
            save_results(drive_train_res_path, "train", log_train)
################################################################################
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        global_steps += 1
################################################################################
        if global_steps % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
        if (global_steps % (eval_steps * gradient_accumulation_steps)) == 0:
            eval_loss, perplexity = evaluate()
            log_eval = {
                    "loss/eval": eval_loss,
                    "perplexity": perplexity,
                    "epoch": epoch,
                    "lr": lr_scheduler.get_lr(),
                    "steps": step,
                    "global_steps" : global_steps,
                    "competed_steps": completed_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
            }
            accelerator.print(log_eval)
            log_train = {
                    "epoch": epoch,
                    "lr": lr_scheduler.get_lr(),
                    "steps": step,
                    "global_steps" : global_steps-1,
                    "competed_steps": completed_steps-1,
                    "loss/train": loss.item() * gradient_accumulation_steps,
            }
            save_results(drive_train_res_path, "train", log_train)
            save_results(drive_train_res_path, "valid", log_eval)
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(model_output_dir, save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(model_output_dir)
                repo.push_to_hub(
                    commit_message=f"Training in progress epoch {epoch}; steps {step}; global_steps {global_steps}; completed_steps {completed_steps}; lr {lr_scheduler.get_lr()}; loss/train {loss.item() * gradient_accumulation_steps}; loss/eval {eval_loss}; perplexity {perplexity}", blocking=False    # train loss is for the next step
                )
            model.train()

################################################################################
################################################################################
################################################################################

#GRADIENT UPDATE (In case (global_steps % gradient_accumulation_steps != 0)
if (global_steps % gradient_accumulation_steps != 0):
    for step, batch in tqdm(
        enumerate(train_dataloader, start=0), total=(gradient_accumulation_steps - (global_steps % gradient_accumulation_steps)) - 1
    ):
        logits = model(batch["input_ids"]).logits
        loss = custom_weighted_loss(batch["input_ids"], logits)
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        global_steps += 1
        if global_steps % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
            break

# FINAL EVALUATE AND SAVE
additional_steps = 0
if (global_steps % gradient_accumulation_steps != 0):
  additional_steps = gradient_accumulation_steps - (global_steps % gradient_accumulation_steps)
with torch.no_grad():
  last_train_loss = 0
  for batch in train_dataloader:
      logits = model(batch["input_ids"]).logits
      loss = custom_weighted_loss(batch["input_ids"], logits)
      last_train_loss = loss.item()
      break
eval_loss, perplexity = evaluate()
log_eval = {
        "loss/eval": eval_loss,
        "perplexity": perplexity,
        "epoch": epoch,
        "lr": lr_scheduler.get_lr(),
        "steps": num_steps_per_epoch + additional_steps,
        "global_steps" : global_steps,
        "competed_steps": completed_steps,
        "loss/train": last_train_loss,
}
accelerator.print(log_eval)
log_train = {
        "epoch": epoch,
        "lr": lr_scheduler.get_lr(),
        "steps": num_steps_per_epoch + additional_steps,
        "global_steps" : global_steps,
        "competed_steps": completed_steps,
        "loss/train": last_train_loss,
}
save_results(drive_train_res_path, "train", log_train)
save_results(drive_train_res_path, "valid", log_eval)
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(model_output_dir, save_function=accelerator.save)
if accelerator.is_main_process:
    tokenizer.save_pretrained(model_output_dir)
    repo.push_to_hub(
        commit_message=f"Final model: epoch {epoch}; steps {num_steps_per_epoch+additional_steps}; global_steps {global_steps}; completed_steps {completed_steps}; lr {lr_scheduler.get_lr()}; loss/train {last_train_loss}; loss/eval {eval_loss}; perplexity {perplexity}", blocking=False
    )
model.train()

torch.cuda.empty_cache()

  0%|          | 0/5579 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


{'epoch': 0, 'lr': [0.0, 0.0], 'steps': 0, 'global_steps': 0, 'competed_steps': 0, 'loss/train': 12.05659294128418}


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.11 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.20 GiB is free. Process 4753 has 13.54 GiB memory in use. Of the allocated memory 10.98 GiB is allocated by PyTorch, and 2.43 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
evaluate()

(11.055583953857422, 63296.4140625)

### TESTING PRETRAINED MODEL

In [ ]:
coq_model = GPT2LMHeadModel.from_pretrained("Andrusyshyn/gpt2-pretrained-for-coq-pt-custom-train")
coq_tokenizer = AutoTokenizer.from_pretrained("Andrusyshyn/gpt2-coq-tokenizer")
pipe = pipeline(
    "text-generation", model=coq_model, tokenizer=coq_tokenizer#, device=0
)

KeyboardInterrupt: 

In [ ]:
model = coq_model
evaluate()

KeyboardInterrupt: 

In [ ]:
txt = """Theorem identity : forall (n : nat), n = n."""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Theorem identity : forall (n : nat), n = n.
intros; red; intros; destruct n; simpl; auto.
Qed.

Theorem eq_sym : forall X : Ens, IN X X H.
intros;


### SANDBOX

In [ ]:
class CustomStoppingCriteria(StoppingCriteria):
    def __init__(self, stop_sequences):
        super().__init__()
        self.stop_sequences = stop_sequences

    def __call__(self, input_ids, scores, **kwargs):
        # print(input_ids)
        # Check if the generated text ends with any of the stop sequences
        for sequence in self.stop_sequences:
            subsequence = input_ids[0][-len(sequence):]
            # print("###############################################################")
            # print(sequence, subsequence)
            if (sequence == subsequence.tolist()):
                # print("RETURN TRUEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE")
                return True  # Stop criteria met
        return False  # Stop criteria not met

In [ ]:
# stop_sequences = [['ĠQed', '.'], ['Qed', '.']]  # Example stop sequences
stop_sequences = [[372, 14]]
custom_stopping_criteria = CustomStoppingCriteria(stop_sequences)
stopping_criteria = StoppingCriteriaList([custom_stopping_criteria])

In [ ]:
for stop_seq in stop_sequences:
  print(tokenizer.encode(stop_seq))

In [ ]:
print(tokenizer.encode('ĠQed'))

[129, 255, 372]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Andrusyshyn/gpt2-coq-tokenizer")
model = GPT2LMHeadModel.from_pretrained("Andrusyshyn/gpt2-pretrained-for-coq-pt-custom-train")
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, stopping_criteria=stopping_criteria#, device=0
)

In [ ]:
# Generate text
generated_text = pipe(["Theorem theorem1 : forall (n : nat), n + 0 = n."], num_return_sequences=50)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[[{'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nProof.\nauto with arith.\nQed.'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nHint Immediate plus_reg_l: arith.'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nProof.\n  induction n as IHn as [ |'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nProof.\nintro n; case n; auto'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nAdmitted. (* QuickChick plus_n_O.'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nintros n; rewrite mult_comm.\nrewrite'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nintros n; elim n; auto.\nQed'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nProof.\nintros. omega.\nQed.'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nintro n; case n; auto.\nsimpl'}, {'generated_text': 'Theorem theorem1 : forall (n : nat), n 

In [ ]:
for _ in range(50):
  generated_text = pipe(["Theorem theorem1 : forall (n : nat), n + 0 = n."], num_return_sequences=1)
  print(generated_text)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[[{'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nintros n; elim n; unfold O in |- *; auto.\nQed.'}]]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[[{'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nsimpl in |- *; auto.\nQed.'}]]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

[[{'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nauto.\nQed.'}]]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[[{'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nProof.\n  induction n; simpl; trivial.\n  destruct 1; trivial.\n  rewrite IHn.\n  rewrite IHn.\n  trivial.\nQed.'}]]


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[[{'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nintro n. \nsymmetry  in |- *. \napply mult_le_compat_l.\napply le_plus_minus.\ninstantiate (1'}]]
[[{'generated_text': 'Theorem theorem1 : forall (n : nat), n + 0 = n.\nProof.\n  intro n.\n  rewrite (plus_comm n).\n  apply plus_le_compat.\n  cset (plus (fact n) n); rewrite'}]]


In [ ]:
text = """Theorem theorem1 : forall (n : nat), n + 0 = n.

    intuition.
    eapply H.
    trivial.
    Qed."""

tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

# text = """Theorem theorem1 : forall (n : nat), n + 0 = n.
# Proof.
#    apply eq_refl.
# Qed."""
# #p_tokenizer(sample["content"], truncation=False)["input_ids"]
# tokenized_text = tokenizer.tokenize(text)
# print(tokenized_text)

['Theorem', 'Ġtheorem', '1', 'Ġ:', 'Ġforall', 'Ġ(', 'n', 'Ġ:', 'Ġnat', '),', 'Ġn', 'Ġ+', 'Ġ0', 'Ġ=', 'Ġn', '.', 'ĊĠĠĠĠĊĠĠĠ', 'Ġintuition', '.', 'ĊĠĠĠ', 'Ġeapply', 'ĠH', '.', 'ĊĠĠĠ', 'Ġtrivial', '.', 'ĊĠĠĠ', 'ĠQed', '.']


In [ ]:
text = """Theorem theorem1 : forall (n : nat), n + 0 = n.

    intuition.
    eapply H.
    trivial.
    Qed."""
input_ids = tokenizer(text, truncation=False)["input_ids"][-2:]
print(input_ids)
print(tokenizer.decode(input_ids, skip_special_tokens=True))

[560, 14]
 Qed.


In [ ]:
print(tokenizer.decode(199) == '\n')

True
